We firstly import all the necessary libraries.

In [ ]:
import json
import re

from datasets import load_dataset
import jsonlines
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    StoppingCriteria,
    StoppingCriteriaList,
)
from tqdm import tqdm
from mxeval.mxeval.evaluation import evaluate_functional_correctness

# import torch_directml
# dml = torch_directml.device()

Next, we obtain the dataset and create `problem_dict` variable. We obtain and download pretrained transformer tokenizer and model `deepseek-coder-1.3b-base`.

In [ ]:
dataset = load_dataset("jetbrains/Kotlin_HumanEval")['train']
problem_dict = {problem['task_id']: problem for problem in dataset}

device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-coder-1.3b-base", trust_remote_code=True).to(device)

In [ ]:
with open("answers", "r") as file:
    line = file.readline()
line

with jsonlines.open("answers1", mode="w") as writer:
    for lines in [line]:
        writer.write(lines)

This part of the code is copied from the hugging face documentation of `jetbrains/Kotlin_HumanEval` dataset. We split the original code into two parts:
- Code for generating completion of the code (cell below)
- Code for evaluating the completed code (cell below the one below)
This was done mainly because of the computational constraints as my machine couldn't handle the whole process together.

In [ ]:
class StoppingCriteriaSub(StoppingCriteria):
    def __init__(self, stops, tokenizer):
        (StoppingCriteria.__init__(self),)
        self.stops = rf"{stops}"
        self.tokenizer = tokenizer

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        last_three_tokens = [int(x) for x in input_ids.data[0][-3:]]
        decoded_last_three_tokens = self.tokenizer.decode(last_three_tokens)

        return bool(re.search(self.stops, decoded_last_three_tokens))


def generate(problem):
    criterion = StoppingCriteriaSub(stops="\n}\n", tokenizer=tokenizer)
    stopping_criteria = StoppingCriteriaList([criterion])
    
    problem = tokenizer.encode(problem, return_tensors="pt").to('cuda')
    sample = model.generate(
        problem,
        max_new_tokens=256,
        min_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        num_beams=1,
        stopping_criteria=stopping_criteria,
    )
    
    answer = tokenizer.decode(sample[0], skip_special_tokens=True)
    return answer


def clean_asnwer(code):
    # Clean comments
    code_without_line_comments = re.sub(r"//.*", "", code)
    code_without_all_comments = re.sub(
        r"/\*.*?\*/", "", code_without_line_comments, flags=re.DOTALL
    )
    #Clean signatures
    lines = code.split("\n")
    for i, line in enumerate(lines):
        if line.startswith("fun "):
            return "\n".join(lines[i + 1:])
            
    return code


problem_dict = {problem['task_id']: problem for problem in dataset}

output = []
for key in tqdm(list(problem_dict.keys()), leave=False):
    problem = problem_dict[key]["prompt"]
    answer = generate(problem)
    answer = clean_asnwer(answer)
    output.append({"task_id": key, "completion": answer, "language": "kotlin"})

output_file = f"all_answers/answers"
with jsonlines.open(output_file, mode="w") as writer:
    for line in output:
        writer.write(line)

We have previously created the file `all_answers/answers` which contains samples where every sample contains `task_id`, `completion` and `language` information. We iterate throguh every sample and for every sample we create new file that actually represents singular version of `all_answers/answers` file - only one sample is present in it. Then, we pass that file as one of the inputs through the `evaluate_functional_correctness` and calculate if the code completion is correct or not. By doing this, every sample is processed one step at a time, which represents much easier task for my local machine.

In [1]:
all_answers = load_dataset("json", data_files="all_answers/answers")['train']
total = 0
correct = 0

for i, entry in enumerate(all_answers):
    try:
        output_filename = f"all_answers/squad_{i + 1}.json"
        problem_dict = {dataset[i]['task_id']: dataset[i]}
        
        with open(output_filename, 'w') as file:
            json.dump(entry, file)

        output_file = output_filename

        evaluate_functional_correctness(
            sample_file=output_file,
            k=[1],
            n_workers=16,
            timeout=15,
            problem_file=problem_dict
        )

        with open(output_file + '_results.jsonl') as fp:
        
            for line in fp:
                sample_res = json.loads(line)
            
                total += 1
                correct += sample_res['passed']

        import os
        os.remove(output_filename)
        os.remove(output_file + '_results.jsonl')
    except:
        import os
        try:
            os.remove(output_filename)
            os.remove(output_file + '_results.jsonl')
        except:
            continue
        continue

print(f'Pass rate: {correct/total}')

c:\Users\Marko.DESKTOP-CS1PADQ\Desktop\necajetbrains\venvv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skip reading problems -- using problem_file (dict) as problems
{'HumanEval_kotlin/32': {'task_id': 'HumanEval_kotlin/32', 'prompt': "/**\n * You are an expert Kotlin programmer, and here is your task.\n * This function takes a list l and returns a list l' such that\n * l' is identical to l in the indices that are not divisible by three, while its values at the indices that are divisible by three are equal\n * to the values of the corresponding indices of l, but sorted.\n * >>> sort_third([1, 2, 3])\n * [1, 2, 3]\n * >>> sort_third([5, 6, 3, 4, 8, 9, 2])\n * [2, 6, 3, 4, 8, 9, 5]\n *\n */\nfun sortThird(l: List<Int>): List<Int> {\n", 'entry_point': 'sortThird', 'test': 'fun main() {\n    var arg00: List<Int> = mutableListOf(1, 2, 3)\n    var x0: List<Int> = sortThird(arg00)\n    var v0: List<Int> = mutableListOf(1, 2, 3)\n    if (x0 != v0) {\n        throw Exception("Exception -- test case 0 did not pass. x0 = " + x0)\n    }\n\n    var arg10: List<Int> = mutableListOf(5, 3, -5, 2, -3, 3

1it [00:00, ?it/s]


Running test suites...


100%|██████████| 1/1 [00:05<00:00,  5.26s/it]


...

Running test suites...


100%|██████████| 1/1 [00:05<00:00,  5.04s/it]


Writing results to all_answers/squad_161.json_results.jsonl...


100%|██████████| 1/1 [00:00<00:00, 998.88it/s]

Pass rate: 0.2847682119205298


After we have processed all the samples, we can calculate the pass rate.

In [2]:
print(f'Pass rate: {correct/total}')

Pass rate: 0.2847682119205298


Great! Now we can start collecting our synthetic dataset.